## 2.1 Getting your data + feature engineering

In [219]:
#import required libraries
import pandas as pd
import datetime
from datetime import datetime, date
from tqdm import tqdm as tq
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import random
import collections
import itertools
import re

In [220]:
#import data and check them
data=pd.read_csv('bank_transactions.csv')
data

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5
...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,8/4/90,M,NEW DELHI,7635.19,18/9/16,184824,799.0
1048563,T1048564,C6459278,20/2/92,M,NASHIK,27311.42,18/9/16,183734,460.0
1048564,T1048565,C6412354,18/5/89,M,HYDERABAD,221757.06,18/9/16,183313,770.0
1048565,T1048566,C6420483,30/8/78,M,VISAKHAPATNAM,10117.87,18/9/16,184706,1000.0


## Data cleaning

### Handle NaN

In [221]:
#drop NaN rows to avoid having customers without some information
data.dropna(inplace=True)

### Handle Customer DOB

In [222]:
#convert date of birth to datetime object
data['CustomerDOB']=pd.to_datetime(data.CustomerDOB, dayfirst=True)

In [223]:
#treat customers with date of birth greater than 2000
data.loc[data.CustomerDOB.dt.year > 2000, 'CustomerDOB'] = data.loc[data.CustomerDOB.dt.year > 2000, 'CustomerDOB'] - pd.DateOffset(years = 100)

In [224]:
# Replace people with DOB as 1800 to mode DOB, since we suppose the dates are incorrect
mode_year=data[data.CustomerDOB.dt.year!=1800].CustomerDOB.mode()
data.loc[data.CustomerDOB.dt.year==1800,'CustomerDOB']=list(mode_year.values)[0]

### Handle Transaction Date

In [225]:
#convert transaction date column to datetime object
data.TransactionDate=pd.to_datetime(data.TransactionDate,dayfirst=True)

### Derive the Age of Customers

In [226]:
#compute the age for each customer ID to have a more relevant information frome tha date of birth
data["AGE"]=((datetime(2016,12,31)-data.CustomerDOB)/np.timedelta64(1, "Y")).astype(int)


In [227]:
data


,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),AGE
0,T1,C5841053,1994-01-10,F,JAMSHEDPUR,17819.05,2016-08-02,143207,25.0,22
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-08-02,141858,27999.0,59
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-08-02,142712,459.0,20
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-08-02,142714,2060.0,43
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-08-02,181156,1762.5,28
...,...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,1990-04-08,M,NEW DELHI,7635.19,2016-09-18,184824,799.0,26
1048563,T1048564,C6459278,1992-02-20,M,NASHIK,27311.42,2016-09-18,183734,460.0,24
1048564,T1048565,C6412354,1989-05-18,M,HYDERABAD,221757.06,2016-09-18,183313,770.0,27
1048565,T1048566,C6420483,1978-08-30,M,VISAKHAPATNAM,10117.87,2016-09-18,184706,1000.0,38


### Transaction Time

In [228]:
#convert transaction time to datetime
data.TransactionTime=data.TransactionTime.apply(lambda x: datetime.strptime(str(x).zfill(6), '%H%M%S'))
#this will create also a date that will be not correspondent to transaction date but since it will be the same for each customer it will not cause issues

In [229]:
#rename the column TransactionAmount(INR) with only TransactionAmount only for the sake of simplicity
data.rename({'TransactionAmount (INR)':'TransactionAmount'}, inplace=True, axis=1)

After this cleaning part we focus on the generation of the suggested features and 20+ more.
## 2. Feature engeenering

### a) Number of transactions

In [230]:
#groupby according to the customer ID and count the number of transactions for each ID
data['NumTransactions']=data.groupby('CustomerID')['CustomerID'].transform('count')

### b) Number of transactions with a balance bigger than 100 INR

In [231]:
#filter for transactions bigger than 100 USD, groupby according to the customer ID and count the number of transactions
data['NumTransactions_100'] = data[data['TransactionAmount'] > 100].groupby(['CustomerID'])['TransactionAmount'].transform('count')

In [232]:
#NaN values will appear when there aren't transactions bigger than 100 USD for certain customers, so we convert them to 0s
data.NumTransactions_100.fillna(0, inplace=True)

### c) Average amount of the transactions

In [233]:
#groupby for CustomerID and compute the mean among the transaction of that IDb
data['AvgTransactions'] = data.groupby(['CustomerID'])['TransactionAmount'].transform('mean')

### d) Average balance

In [234]:
#groupby as usual and compute the mean among the balances related to a certain customer ID
data['AvgBalance'] = data.groupby(['CustomerID'])['CustAccountBalance'].transform('mean')

### e) Average difference between the balance and the transaction amount for each customer, known as utilisation

In [235]:
#create a column with utilisation that then will be dropped
data['Utilisation']=data['CustAccountBalance'] - data['TransactionAmount']

In [236]:
#compute the mean of utilisation
data['AvgUtilisation'] = data.groupby(['CustomerID'])['Utilisation'].transform('mean')

In [237]:
#drop the utilisation column because is not needed anymore 
data.drop('Utilisation', axis=1, inplace=True)

### f) Most common gender of the customer

In [239]:
#compute the modal gender for each customer ID
data['Gender']=data.groupby(['CustomerID'])['CustGender'].transform(lambda x: pd.Series.mode(x).iloc[0])

### h) Most common location of the customer

In [240]:
#keep only the last word of the string in order to have uniform locations (the loss of information is minimal)
data['CustLocation']=data['CustLocation'].apply(lambda x: str(x).split()[-1])
#locations will end up in regions more than towns (for example we will have only Dehli and not New Delhi but the region will stay the same)

In [241]:
#compute the modal value location for each customer
data['Location']=data.groupby(['CustomerID'])['CustLocation'].transform(lambda x: pd.Series.mode(x).iloc[0])

In [242]:
data

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount,AGE,NumTransactions,NumTransactions_100,AvgTransactions,AvgBalance,AvgUtilisation,Gender,Location
0,T1,C5841053,1994-01-10,F,JAMSHEDPUR,17819.05,2016-08-02,1900-01-01 14:32:07,25.0,22,1,0.0,25.0,17819.050,17794.050,F,JAMSHEDPUR
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-08-02,1900-01-01 14:18:58,27999.0,59,1,1.0,27999.0,2270.690,-25728.310,M,JHAJJAR
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-08-02,1900-01-01 14:27:12,459.0,20,1,1.0,459.0,17874.440,17415.440,F,MUMBAI
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-08-02,1900-01-01 14:27:14,2060.0,43,2,2.0,1469.2,434849.495,433380.295,F,MUMBAI
4,T5,C9031234,1988-03-24,F,MUMBAI,6714.43,2016-08-02,1900-01-01 18:11:56,1762.5,28,1,1.0,1762.5,6714.430,4951.930,F,MUMBAI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,1990-04-08,M,DELHI,7635.19,2016-09-18,1900-01-01 18:48:24,799.0,26,1,1.0,799.0,7635.190,6836.190,M,DELHI
1048563,T1048564,C6459278,1992-02-20,M,NASHIK,27311.42,2016-09-18,1900-01-01 18:37:34,460.0,24,1,1.0,460.0,27311.420,26851.420,M,NASHIK
1048564,T1048565,C6412354,1989-05-18,M,HYDERABAD,221757.06,2016-09-18,1900-01-01 18:33:13,770.0,27,2,2.0,489.0,112719.330,112230.330,M,HYDERABAD
1048565,T1048566,C6420483,1978-08-30,M,VISAKHAPATNAM,10117.87,2016-09-18,1900-01-01 18:47:06,1000.0,38,1,1.0,1000.0,10117.870,9117.870,M,VISAKHAPATNAM


## 3. Generate at least 20 more features

## Features generated from transaction amount

### 1. Minimum value transaction for each ID
To get to know more customers it could be useful to know the extremes of their transactions, in addition to the mean.

In [243]:
#compute the minimum amount transaction for each ID
data['MinTransaction']=data.groupby(['CustomerID'])['TransactionAmount'].transform('min')

### 2. Maximum value transaction for each ID

In [244]:
#repeat for the maximum
data['MaxTransaction']=data.groupby(['CustomerID'])['TransactionAmount'].transform('max')

We see that in the case of only one transaction the minimum and the maximum coincide.

### 3. Median of the transaction amount
Since mean is influenced by extreme values it could be useful to observe the median of the transaction amount.

In [245]:
#compute the median of transaction amount for each ID
data['MdnTransaction']=data.groupby(['CustomerID'])['TransactionAmount'].transform('median')

### 4. Number of transactions greater than 10000 INR
This feature could help to understand if customers realize small or huge amount transactions to better understand the usage of their bank account. We observe that Approximate 2% of the people are involved in transactions with an amount greater than 10000 INR.

In [246]:
#compute the percentage of transaction greater than 10000 INR
data[data.TransactionAmount>10000].TransactionAmount.count()*100/len(data)

2.24804966139088

In [247]:
#filter the dataset for transactions greater than 10000 INR
data['NumTransactions_10000'] = data[data['TransactionAmount'] > 10000].groupby(['CustomerID'])['TransactionAmount'].transform('count')

In [248]:
#substitute NA values with 0s
data.NumTransactions_10000.fillna(0,inplace=True)
data

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount,AGE,NumTransactions,NumTransactions_100,AvgTransactions,AvgBalance,AvgUtilisation,Gender,Location,MinTransaction,MaxTransaction,MdnTransaction,NumTransactions_10000
0,T1,C5841053,1994-01-10,F,JAMSHEDPUR,17819.05,2016-08-02,1900-01-01 14:32:07,25.0,22,1,0.0,25.0,17819.050,17794.050,F,JAMSHEDPUR,25.0,25.0,25.0,0.0
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-08-02,1900-01-01 14:18:58,27999.0,59,1,1.0,27999.0,2270.690,-25728.310,M,JHAJJAR,27999.0,27999.0,27999.0,1.0
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-08-02,1900-01-01 14:27:12,459.0,20,1,1.0,459.0,17874.440,17415.440,F,MUMBAI,459.0,459.0,459.0,0.0
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-08-02,1900-01-01 14:27:14,2060.0,43,2,2.0,1469.2,434849.495,433380.295,F,MUMBAI,878.4,2060.0,1469.2,0.0
4,T5,C9031234,1988-03-24,F,MUMBAI,6714.43,2016-08-02,1900-01-01 18:11:56,1762.5,28,1,1.0,1762.5,6714.430,4951.930,F,MUMBAI,1762.5,1762.5,1762.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,1990-04-08,M,DELHI,7635.19,2016-09-18,1900-01-01 18:48:24,799.0,26,1,1.0,799.0,7635.190,6836.190,M,DELHI,799.0,799.0,799.0,0.0
1048563,T1048564,C6459278,1992-02-20,M,NASHIK,27311.42,2016-09-18,1900-01-01 18:37:34,460.0,24,1,1.0,460.0,27311.420,26851.420,M,NASHIK,460.0,460.0,460.0,0.0
1048564,T1048565,C6412354,1989-05-18,M,HYDERABAD,221757.06,2016-09-18,1900-01-01 18:33:13,770.0,27,2,2.0,489.0,112719.330,112230.330,M,HYDERABAD,208.0,770.0,489.0,0.0
1048565,T1048566,C6420483,1978-08-30,M,VISAKHAPATNAM,10117.87,2016-09-18,1900-01-01 18:47:06,1000.0,38,1,1.0,1000.0,10117.870,9117.870,M,VISAKHAPATNAM,1000.0,1000.0,1000.0,0.0


## Getting Features from the Date of the Transactions:


### 5. The day of the month for transaction
### 6. The month of year in which the transaction was made
### 7. The weekday number on which the transaction was made


These features can help us understand the use of the bank account over time, extracting as much information as possible from the various time levels (eg. year, day of the week...).

In [249]:
#compute the transaction day, month and weekday from TransactionDate, previously parsed with datetime
data["Trans_day"]=data.TransactionDate.apply(lambda x:x.day)

data["Trans_month"]=data.TransactionDate.apply(lambda x:x.month)

data["Trans_weekday"]=data.TransactionDate.apply(lambda x:x.dayofweek)


### 8. Recency: Average of TransactionDates
This feature better indicates when account owners do their transactions during the year.

In [250]:
#compute the number of days from the starting of the year at which the transaction was made
def time_interval_transaction(series):
    days=[]
    for i in series:
  
        delta=i-datetime(2016,1,1,0,0,0)
        days.append(delta.days)
    return sum(days)/len(days)

In [251]:
#apply the function to the groupby object
data['Recency']= data.groupby(['CustomerID'])['TransactionDate'].transform(time_interval_transaction)

### 9. Number of transactions made in the first part of the year 

We want to observe the development of transactions per customer ID in the first part of the year by counting them and calculating their amount. Specularly, we also get information on the trend in the other half of the year.

In [252]:
#count transactions in the first part of the year for each customer
data['NumFirstHalfYear'] = data[data['TransactionDate'] < '2016-06-30'].groupby(['CustomerID'])['TransactionDate'].transform('count')

In [253]:
#substitute NaN values with with zeros
data.NumFirstHalfYear.fillna(0, inplace=True)

### 10. Amount of transactions in the first part of the year

In [254]:
#determine the amount of transactions fro each customer ID
data['AmountFirstHalf'] = data[data['TransactionDate'] < '2016-06-30'].groupby(['CustomerID'])['TransactionAmount'].transform('sum')

In [255]:
#substitute NaN values with with zeros
data.AmountFirstHalf.fillna(0, inplace=True)

## Features derived from customer Age
These features could help us to better understand the composition of the group of people (we suppose family in most of the cases), using the bank account associated with each ID.

### 11. Mean age for Customer ID


In [256]:
#compute the mean age
data['AvgAge'] = data.groupby(['CustomerID'])['AGE'].transform('mean').astype(int)

### 12. Minimum and maximum age
To detect extreme values and compare them.

In [257]:
#compute the minimum age value and cast it as int
data['MinAge'] = data.groupby(['CustomerID'])['AGE'].transform('min').astype(int)

In [258]:
#compute the maximum age and cast it at int
data['MaxAge']=data.groupby(['CustomerID'])['AGE'].transform('max').astype(int)

### 13. Number of transactions of people with more than 40 years
This feature allows us to understand the composition in terms of people over 40, i.e. adults who have access to the account.

In [259]:
#we filter the dataset for transactions belonging to customers of more than 40 years old and then count the number of transactions made
data['NumTransactions_40'] = data[data['AGE'] > 40].groupby(['CustomerID'])['TransactionAmount'].transform('count')

In [260]:
#subsitute NA with zeros
data['NumTransactions_40'].fillna(0,inplace=True)

## Features based on transaction time
To understand the trend in the use of the account for transactions at different times of the day.

### 14. Transactions which took place in the different parts of the day (Morning, Afternoon, Evening, Night).


In [261]:
#compute the function that encodes the different parts of the day
def transaction_time(series):
    if series.hour>=6 and series.hour<13: # Morning is encoded as 1
        return 1
    elif series.hour >= 13 and series.hour<=17: # Afternoon is encoded as 2
        return 2
    elif series.hour > 17 and series.hour <= 22: ## Evening is encoded as 3
        return 3
    elif series.hour > 22:  # Night is encoded as 4
        return 4
    elif series.hour < 6:   
        return 5

In [262]:
#apply it to TransactionTime
data['Transaction_part_day']=data.TransactionTime.apply(lambda x:transaction_time(x))

## Features based on Customer Account Balance
These features are useful as they allow us to understand the variability of the account balance of customers and thus understand their status. 

### 15. Median Balance
As usual since median is not eccessively influenced as mean by extreme values.

In [263]:
#groupby as usual and compute the median among the balances related to a certain customer ID
data['MdnBalance'] = data.groupby(['CustomerID'])['CustAccountBalance'].transform('median')

### 16. Balance greater than 100K INR

In [264]:
#filter the dataset for balances greater than 100K INR and then count
data['Balance_greater_100k']=data[data['CustAccountBalance']>100000].groupby(['CustomerID'])['CustAccountBalance'].transform('count')

In [265]:
#fill NA with zeros
data['Balance_greater_100k'].fillna(0,inplace=True)

### 17. Ratio of Transaction amount/ Balance in account
We create a new feature based for each customer, based on the average amount of transaction and average amount of balance.The aim of this feature to see the patterns in their transactions amount based on their account balance and to observe the management of the account in terms of expenditure in relation to how much is present in the account as balance.



In [266]:
#compute the ratio between the transaction amount and the balance present in the account
data['Tr_B_ratio'] = data.groupby(['CustomerID'])['TransactionAmount'].transform('mean')/ (data.groupby(['CustomerID'])['CustAccountBalance'].transform('mean')+1)

## Features based on Locations
Here we calculate for each customer how many location he had visited to make payments using his bank's services.This might be 
useful to differentiate between customers who are frequent travellers against the rest.

### 18. Numer of locations made in different places

In [267]:
#count the number of transactions made in different locations
data['Number_Locations']=data.groupby(['CustomerID'])['CustLocation'].transform('count')

### 19. Average Amount spent per location

In [268]:
#compute the average transaction amount spent for each location
data['Average_amount_location']=data.groupby('CustomerID')['TransactionAmount'].transform('mean')/data.groupby('CustomerID')['CustLocation'].transform('count')

In [269]:
data

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount,AGE,NumTransactions,NumTransactions_100,AvgTransactions,AvgBalance,AvgUtilisation,Gender,Location,MinTransaction,MaxTransaction,MdnTransaction,NumTransactions_10000,Trans_day,Trans_month,Trans_weekday,Recency,NumFirstHalfYear,AmountFirstHalf,AvgAge,MinAge,MaxAge,NumTransactions_40,Transaction_part_day,MdnBalance,Balance_greater_100k,Tr_B_ratio,Number_Locations,Average_amount_location
0,T1,C5841053,1994-01-10,F,JAMSHEDPUR,17819.05,2016-08-02,1900-01-01 14:32:07,25.0,22,1,0.0,25.0,17819.050,17794.050,F,JAMSHEDPUR,25.0,25.0,25.0,0.0,2,8,1,214.0,0.0,0.0,22,22,22,0.0,2,17819.050,0.0,0.001403,1,25.0
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-08-02,1900-01-01 14:18:58,27999.0,59,1,1.0,27999.0,2270.690,-25728.310,M,JHAJJAR,27999.0,27999.0,27999.0,1.0,2,8,1,214.0,0.0,0.0,59,59,59,1.0,2,2270.690,0.0,12.325185,1,27999.0
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-08-02,1900-01-01 14:27:12,459.0,20,1,1.0,459.0,17874.440,17415.440,F,MUMBAI,459.0,459.0,459.0,0.0,2,8,1,214.0,0.0,0.0,20,20,20,0.0,2,17874.440,0.0,0.025678,1,459.0
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-08-02,1900-01-01 14:27:14,2060.0,43,2,2.0,1469.2,434849.495,433380.295,F,MUMBAI,878.4,2060.0,1469.2,0.0,2,8,1,219.5,0.0,0.0,40,37,43,1.0,2,434849.495,1.0,0.003379,2,734.6
4,T5,C9031234,1988-03-24,F,MUMBAI,6714.43,2016-08-02,1900-01-01 18:11:56,1762.5,28,1,1.0,1762.5,6714.430,4951.930,F,MUMBAI,1762.5,1762.5,1762.5,0.0,2,8,1,214.0,0.0,0.0,28,28,28,0.0,3,6714.430,0.0,0.262455,1,1762.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,1990-04-08,M,DELHI,7635.19,2016-09-18,1900-01-01 18:48:24,799.0,26,1,1.0,799.0,7635.190,6836.190,M,DELHI,799.0,799.0,799.0,0.0,18,9,6,261.0,0.0,0.0,26,26,26,0.0,3,7635.190,0.0,0.104633,1,799.0
1048563,T1048564,C6459278,1992-02-20,M,NASHIK,27311.42,2016-09-18,1900-01-01 18:37:34,460.0,24,1,1.0,460.0,27311.420,26851.420,M,NASHIK,460.0,460.0,460.0,0.0,18,9,6,261.0,0.0,0.0,24,24,24,0.0,3,27311.420,0.0,0.016842,1,460.0
1048564,T1048565,C6412354,1989-05-18,M,HYDERABAD,221757.06,2016-09-18,1900-01-01 18:33:13,770.0,27,2,2.0,489.0,112719.330,112230.330,M,HYDERABAD,208.0,770.0,489.0,0.0,18,9,6,259.5,0.0,0.0,37,27,47,0.0,3,112719.330,1.0,0.004338,2,244.5
1048565,T1048566,C6420483,1978-08-30,M,VISAKHAPATNAM,10117.87,2016-09-18,1900-01-01 18:47:06,1000.0,38,1,1.0,1000.0,10117.870,9117.870,M,VISAKHAPATNAM,1000.0,1000.0,1000.0,0.0,18,9,6,261.0,0.0,0.0,38,38,38,0.0,3,10117.870,0.0,0.098825,1,1000.0


To complete the picture on the features, we calculate this last mixed feature and in the process create the reduced dataset, removing the original features so that we have the final ones that can be subjected to the dimensional reduction analysis.

## Features based on transaction amount and time

As the 14th feature which could help to understand the transaction trend during the day for each costumer ID this one tries to get whether the amount of transactions varies with respect to different times of the day (eg. morning, afternoon).

### 20. Average amount of Transactions taking place in Morning and Afternoon



In [270]:
#define a function to identify transaction made in the first part of the day or the second
def trans_morn_afternoon(series):
    trans=0
    location=0

    for i in series.iterrows():
        if (transaction_time(i[1].TransactionTime)==1) or (transaction_time(i[1].TransactionTime)==2) :
            trans+=i[1].TransactionAmount
            location+=1
            
    #handle the situation in which no transaction took place neither in the morning nor in the afteroon       
    if location==0:
        return 0
    else:
       return trans/location

In [271]:
#apply the function to the groupby object without creating a new column otherwise we will obtain only NA values
x=data.groupby('CustomerID')['TransactionAmount','TransactionTime'].apply(trans_morn_afternoon)


Then we removed all the columns related to no needed features, dropping them to obtain a reduced dataset cointaining only the features of interest.

In [272]:
#remove original columns
data_reduced=data.drop(['TransactionID','CustomerDOB','CustGender','CustLocation','CustAccountBalance','MdnTransaction','AGE','TransactionDate','TransactionTime','TransactionAmount'],axis=1).copy()

In [273]:
dict2={}
for i in data_reduced.columns:
    if i!='CustomerID':
        dict2[i]=['mean']
dict2['Location']=['first']
dict2['Gender']=['first']

In [274]:
data_reduced=data_reduced.groupby('CustomerID').agg(dict2)
#remove row index to have as index the CustomerID
data_reduced.columns=data_reduced.columns.droplevel(1)
#add to the reduced dataset the column with the last feature
data_reduced['Trans_morn_afternoon']=x

In [277]:
pd.set_option('max_columns', 10)
data_reduced

,NumTransactions,NumTransactions_100,AvgTransactions,AvgBalance,AvgUtilisation,...,Balance_greater_100k,Tr_B_ratio,Number_Locations,Average_amount_location,Trans_morn_afternoon
CustomerID,,,,,,,,,,,
C1010011,2,2.0,2553.0,76340.635,73787.635,...,0.5,0.033442,2,1276.50,4750.0
C1010012,1,1.0,1499.0,24204.490,22705.490,...,0.0,0.061928,1,1499.00,0.0
C1010014,2,2.0,727.5,100112.950,99385.450,...,0.5,0.007267,2,363.75,1205.0
C1010018,1,0.0,30.0,496.180,466.180,...,0.0,0.060340,1,30.00,30.0
C1010024,1,1.0,5000.0,87058.650,82058.650,...,0.0,0.057432,1,5000.00,5000.0
...,...,...,...,...,...,...,...,...,...,...,...
C9099836,1,1.0,691.0,133067.230,132376.230,...,1.0,0.005193,1,691.00,0.0
C9099877,1,1.0,222.0,96063.460,95841.460,...,0.0,0.002311,1,222.00,222.0
C9099919,1,1.0,126.0,5559.750,5433.750,...,0.0,0.022659,1,126.00,126.0
